![logog](https://raw.githubusercontent.com/Pacific-AI-Corp/langtest/main/docs/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Pacific-AI-Corp/langtest/blob/main/demo/tutorials/misc/Multiple_dataset.ipynb)

**LangTest** is an open-source python library designed to help developers deliver safe and effective Natural Language Processing (NLP) models. Whether you are using **John Snow Labs, Hugging Face, Spacy** models or **OpenAI, Cohere, AI21, Hugging Face Inference API and Azure-OpenAI** based LLMs, it has got you covered. You can test any Named Entity Recognition (NER), Text Classification, fill-mask, Translation model using the library. We also support testing LLMS for Question-Answering, Summarization and text-generation tasks on benchmark datasets. The library supports 60+ out of the box tests. For a complete list of supported test categories, please refer to the [documentation](http://langtest.org/docs/pages/docs/test_categories).

Metrics are calculated by comparing the model's extractions in the original list of sentences against the extractions carried out in the noisy list of sentences. The original annotated labels are not used at any point, we are simply comparing the model against itself in a 2 settings.

# Getting started with LangTest

In [ ]:
!pip install "langtest[openai,transformers,evaluate]"

# Harness and Its Parameters

The Harness class is a testing class for Natural Language Processing (NLP) models. It evaluates the performance of a NLP model on a given task using test data and generates a report with test results.Harness can be imported from the LangTest library in the following way.

In [1]:
# Import Harness from the LangTest library
from langtest import Harness

It imports the Harness class from within the module, that is designed to provide a blueprint or framework for conducting NLP testing, and that instances of the Harness class can be customized or configured for different testing scenarios or environments.

Here is a list of the different parameters that can be passed to the Harness function:

<br/>


| Parameter  | Description |  
| - | - |
|**task**     |Task for which the model is to be evaluated (question-answering or summarization)|
| **model**     | Specifies the model(s) to be evaluated. This parameter can be provided as either a dictionary or a list of dictionaries. Each dictionary should contain the following keys: <ul><li>model (mandatory): 	PipelineModel or path to a saved model or pretrained pipeline/model from hub.</li><li>hub (mandatory): Hub (library) to use in back-end for loading model from public models hub or from path</li></ul>|
| **data**      | The data to be used for evaluation. A dictionary providing flexibility and options for data sources. It should include the following keys: <ul><li>data_source (mandatory): The source of the data.</li><li>subset (optional): The subset of the data.</li><li>feature_column (optional): The column containing the features.</li><li>target_column (optional): The column containing the target labels.</li><li>split (optional): The data split to be used.</li><li>source (optional): Set to 'huggingface' when loading Hugging Face dataset.</li></ul> |
| **config**    | Configuration for the tests to be performed, specified in the form of a YAML file. |

<br/>
<br/>

# OpenAI Model Testing For Question Answering

In this section, we dive into testing of OpenAI models in Question Answering task.

LangTest supports robustness tests for LLM testing for now.

### Set environment for OpenAI

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"

### Multi Dataset Testing

In order to evaluate the model's performance on multiple datasets, we can utilize a Jupyter notebook and provide a list of dictionaries to the `data` parameter. Each dictionary within the list should contain the following keys:

```
data=[
    {"data_source": "BoolQ", "split": "test-tiny"},
    {"data_source": "NQ-open", "split": "test-tiny"},
    {"data_source": "MedQA", "split": "test-tiny"},
    {"data_source": "LogiQA", "split": "test-tiny"},
],
```

Here, we specify different data sources and their corresponding splits for testing. This allows for a comprehensive evaluation of the model's performance across diverse datasets. The notebook can then be executed to assess how well the model generalizes to various types of questions and contexts presented in these datasets.

In [3]:
harness = Harness(
    task="question-answering",
    model={"model": "gpt-3.5-turbo-instruct", "hub": "openai"},
    data=[
        {"data_source": "BoolQ", "split": "test-tiny"},
        {"data_source": "NQ-open", "split": "test-tiny"},
        {"data_source": "MedQA", "split": "test-tiny"},
        {"data_source": "LogiQA", "split": "test-tiny"},
    ],
)

Test Configuration : 
 {
 "model_parameters": {
  "max_tokens": 64
 },
 "tests": {
  "defaults": {
   "min_pass_rate": 1.0
  },
  "robustness": {
   "add_typo": {
    "min_pass_rate": 0.7
   },
   "lowercase": {
    "min_pass_rate": 0.7
   }
  }
 }
}


## Robustness

For tests we used uppercase, Dyslexia Word Swap, Add Slangs, Insert Abbreviations and Speech to Text typos . Other available robustness tests for QA task are:
* `add_context`
* `add_contraction`
* `add_punctuation`
* `add_typo`
* `add_ocr_typo`
* `american_to_british`
* `british_to_american`
* `lowercase`
* `strip_punctuation`
* `titlecase`
* `uppercase`
* `number_to_word`
* `add_abbreviation`
* `add_speech_to_text_typo`
* `add_slangs`
* `dyslexia_word_swap`
* `multiple_perturbations`
* `adjective_synonym_swap`
* `adjective_antonym_swap`
* `strip_all_punctuation`

You can also set prompts and other model parameters in config. Possible parameters are:
* `user_promt:` Promt to be given to the model.
* `temperature:` Temperature of the model.
* `max_tokens:` Maximum number of output tokens allowed for model.

In [4]:
harness.configure(
    {
        "tests": {
            "defaults": {"min_pass_rate": 0.65},
            "robustness": {
                "uppercase": {"min_pass_rate": 0.66},
                "dyslexia_word_swap": {"min_pass_rate": 0.60},
                "add_abbreviation": {"min_pass_rate": 0.60},
                "add_slangs": {"min_pass_rate": 0.60},
                "add_speech_to_text_typo": {"min_pass_rate": 0.60},
            },
        }
    }
)

{'tests': {'defaults': {'min_pass_rate': 0.65},
  'robustness': {'uppercase': {'min_pass_rate': 0.66},
   'dyslexia_word_swap': {'min_pass_rate': 0.6},
   'add_abbreviation': {'min_pass_rate': 0.6},
   'add_slangs': {'min_pass_rate': 0.6},
   'add_speech_to_text_typo': {'min_pass_rate': 0.6}}}}

➤ You can adjust the level of transformation in the sentence by using the "`prob`" parameter, which controls the proportion of words to be changed during robustness tests.

➤ **NOTE** : "`prob`" defaults to 1.0, which means all words will be transformed.
```
harness.configure(
{
 'tests': {
    'defaults': {'min_pass_rate': 0.65},
      'robustness': {
        'uppercase': {'min_pass_rate': 0.66, 'prob': 0.50},
        'dyslexia_word_swap':{'min_pass_rate': 0.60, 'prob': 0.70},
      }
  }
})

```

Here we have configured the harness to perform Five robustness tests and defined the minimum pass rate for each test.

In [6]:
#slice the data
harness.data = {k: v[:5] for k, v in harness.data.items()}

### Generating the test cases.

In [7]:
harness.generate()

                                     BoolQ                                      


Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]
[W010] - Test 'add_slangs': 3 samples removed out of 5



--------------------------------------------------------------------------------

                                    NQ-open                                     


Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]
[W010] - Test 'dyslexia_word_swap': 1 samples removed out of 5
[W010] - Test 'add_abbreviation': 1 samples removed out of 5
[W010] - Test 'add_slangs': 4 samples removed out of 5



--------------------------------------------------------------------------------

                                     MedQA                                      


Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]


--------------------------------------------------------------------------------

                                     LogiQA                                     


Generating testcases...: 100%|██████████| 1/1 [00:00<?, ?it/s]
[W010] - Test 'add_slangs': 1 samples removed out of 5



--------------------------------------------------------------------------------



In [8]:
harness.testcases()

,category,dataset_name,test_type,original_context,original_question,perturbed_context,perturbed_question,options
0,robustness,BoolQ,uppercase,20 euro note -- Until now there has been only ...,is the first series 20 euro note still legal t...,20 EURO NOTE -- UNTIL NOW THERE HAS BEEN ONLY ...,IS THE FIRST SERIES 20 EURO NOTE STILL LEGAL T...,-
1,robustness,BoolQ,uppercase,2018–19 UEFA Champions League -- The final wil...,do the champions league winners get automatic ...,2018–19 UEFA CHAMPIONS LEAGUE -- THE FINAL WIL...,DO THE CHAMPIONS LEAGUE WINNERS GET AUTOMATIC ...,-
2,robustness,BoolQ,uppercase,Bullsnake -- Bullsnakes are very powerful cons...,can a bull snake kill a small dog,BULLSNAKE -- BULLSNAKES ARE VERY POWERFUL CONS...,CAN A BULL SNAKE KILL A SMALL DOG,-
3,robustness,BoolQ,uppercase,NBA playoffs -- All rounds are best-of-seven s...,are all nba playoff games best of 7,NBA PLAYOFFS -- ALL ROUNDS ARE BEST-OF-SEVEN S...,ARE ALL NBA PLAYOFF GAMES BEST OF 7,-
4,robustness,BoolQ,uppercase,Manchester station group -- The Manchester sta...,can i use my train ticket on the tram in manch...,MANCHESTER STATION GROUP -- THE MANCHESTER STA...,CAN I USE MY TRAIN TICKET ON THE TRAM IN MANCH...,-
...,...,...,...,...,...,...,...,...
85,robustness,LogiQA,add_speech_to_text_typo,In the planning of a new district in a townshi...,"Based on the above statement, which of the fol...",In the planning of Ae new district in a townsh...,"Based Aune the above statement, which of the f...",A. Civic Park is north of the administrative s...
86,robustness,LogiQA,add_speech_to_text_typo,The company sent three young staff members to ...,So what are the three young people on business?\n,The company Scent three young staff members to...,So what Er the three young people on business?\n,"A. 0-year-old accountant, 20-year-old salesper..."
87,robustness,LogiQA,add_speech_to_text_typo,"In a traditional Chinese medicine preparation,...","According to the above statement, which of the...",Inn a traditional Chinese medicine preparation...,"According to the above statement, which of the...",A. o dangshen.\nB. o Shouwu.\nC. 白 术.\nD. 白 术.
88,robustness,LogiQA,add_speech_to_text_typo,"In recent years, graduate entrance examination...",Which of the following can best strengthen the...,"Inn recent years, graduate entrance examinatio...",Which of the following can best strengthen the...,"A. If you take an English tutoring class, you ..."


harness.generate() method automatically generates the test cases (based on the provided configuration)

### Running the tests

In [9]:
harness.run()

                                     BoolQ                                      


Running testcases... : 100%|██████████| 22/22 [00:17<00:00,  1.29it/s]


--------------------------------------------------------------------------------

                                    NQ-open                                     


Running testcases... : 100%|██████████| 19/19 [00:22<00:00,  1.21s/it]


--------------------------------------------------------------------------------

                                     MedQA                                      


Running testcases... : 100%|██████████| 25/25 [00:18<00:00,  1.33it/s]


--------------------------------------------------------------------------------

                                     LogiQA                                     


Running testcases... : 100%|██████████| 24/24 [00:18<00:00,  1.30it/s]

--------------------------------------------------------------------------------



Called after harness.generate() and is to used to run all the tests.  Returns a pass/fail flag for each test.

### Generated Results

In [10]:
harness.generated_results()

,category,dataset_name,test_type,original_context,original_question,perturbed_context,perturbed_question,options,expected_result,actual_result,pass
0,robustness,BoolQ,uppercase,20 euro note -- Until now there has been only ...,is the first series 20 euro note still legal t...,20 EURO NOTE -- UNTIL NOW THERE HAS BEEN ONLY ...,IS THE FIRST SERIES 20 EURO NOTE STILL LEGAL T...,-,\n\nFalse,\n\nFalse,True
1,robustness,BoolQ,uppercase,2018–19 UEFA Champions League -- The final wil...,do the champions league winners get automatic ...,2018–19 UEFA CHAMPIONS LEAGUE -- THE FINAL WIL...,DO THE CHAMPIONS LEAGUE WINNERS GET AUTOMATIC ...,-,\n\nTrue,\n\nTrue,True
2,robustness,BoolQ,uppercase,Bullsnake -- Bullsnakes are very powerful cons...,can a bull snake kill a small dog,BULLSNAKE -- BULLSNAKES ARE VERY POWERFUL CONS...,CAN A BULL SNAKE KILL A SMALL DOG,-,\n\nFalse,\n\nFalse,True
3,robustness,BoolQ,uppercase,NBA playoffs -- All rounds are best-of-seven s...,are all nba playoff games best of 7,NBA PLAYOFFS -- ALL ROUNDS ARE BEST-OF-SEVEN S...,ARE ALL NBA PLAYOFF GAMES BEST OF 7,-,\n\nTrue,\n\nTrue,True
4,robustness,BoolQ,uppercase,Manchester station group -- The Manchester sta...,can i use my train ticket on the tram in manch...,MANCHESTER STATION GROUP -- THE MANCHESTER STA...,CAN I USE MY TRAIN TICKET ON THE TRAM IN MANCH...,-,\n\nTrue,\n\nTrue,True
...,...,...,...,...,...,...,...,...,...,...,...
85,robustness,LogiQA,add_speech_to_text_typo,In the planning of a new district in a townshi...,"Based on the above statement, which of the fol...",In the planning of Ae new district in a townsh...,"Based Aune the above statement, which of the f...",A. Civic Park is north of the administrative s...,B. The leisure area is southwest of the cultu...,D. The business district is southeast of the ...,False
86,robustness,LogiQA,add_speech_to_text_typo,The company sent three young staff members to ...,So what are the three young people on business?\n,The company Scent three young staff members to...,So what Er the three young people on business?\n,"A. 0-year-old accountant, 20-year-old salesper...","C. 4-year-old accountant, 20-year-old salespe...","D. 0-year-old accountant, 20-year-old account...",True
87,robustness,LogiQA,add_speech_to_text_typo,"In a traditional Chinese medicine preparation,...","According to the above statement, which of the...",Inn a traditional Chinese medicine preparation...,"According to the above statement, which of the...",A. o dangshen.\nB. o Shouwu.\nC. 白 术.\nD. 白 术.,B. Shouwu.,B. Shouwu.,True
88,robustness,LogiQA,add_speech_to_text_typo,"In recent years, graduate entrance examination...",Which of the following can best strengthen the...,"Inn recent years, graduate entrance examinatio...",Which of the following can best strengthen the...,"A. If you take an English tutoring class, you ...","A. If you take an English tutoring class, you...","A. If you take an English tutoring class, you...",True


This method returns the generated results in the form of a pandas dataframe, which provides a convenient and easy-to-use format for working with the test results. You can use this method to quickly identify the test cases that failed and to determine where fixes are needed.

### Final Results

We can call `.report()` which summarizes the results giving information about pass and fail counts and overall test pass/fail flag.

In [11]:
harness.report()

Benchmarking Results: gpt-3.5-turbo-instruct  \
                                                                                  fail_count   
dataset_name category   test_type                                                              
BoolQ        robustness uppercase                                                          0   
                        dyslexia_word_swap                                                 1   
                        add_abbreviation                                                   0   
                        add_slangs                                                         1   
                        add_speech_to_text_typo                                            0   
NQ-open      robustness uppercase                                                          1   
                        dyslexia_word_swap                                                 0   
                        add_abbreviation                                                   1   
                        add_slangs                                                         1   
                        add_speech_to_text_typo                                            4   
MedQA        robustness uppercase                                                          2   
                        dyslexia_word_swap                                                 1   
                        add_abbreviation                                                   2   
                        add_slangs                                                         2   
                        add_speech_to_text_typo                                            2   
LogiQA       robustness uppercase                                                          2   
                        dyslexia_word_swap                                                 1   
                        add_abbreviation                                                   2   
                        add_slangs                                                         1   
                        add_speech_to_text_typo                                            1   

                                                                      \
                                                pass_count pass_rate   
dataset_name category   test_type                                      
BoolQ        robustness uppercase                        5      100%   
                        dyslexia_word_swap               4       80%   
                        add_abbreviation                 5      100%   
                        add_slangs                       1       50%   
                        add_speech_to_text_typo          5      100%   
NQ-open      robustness uppercase                        4       80%   
                        dyslexia_word_swap               4      100%   
                        add_abbreviation                 3       75%   
                        add_slangs                       0        0%   
                        add_speech_to_text_typo          1       20%   
MedQA        robustness uppercase                        3       60%   
                        dyslexia_word_swap               4       80%   
                        add_abbreviation                 3       60%   
                        add_slangs                       3       60%   
                        add_speech_to_text_typo          3       60%   
LogiQA       robustness uppercase                        3       60%   
                        dyslexia_word_swap               4       80%   
                        add_abbreviation                 3       60%   
                        add_slangs                       3       75%   
                        add_speech_to_text_typo          4       80%   

                                                                          
                                                minimum_pass_rate   pass  
dataset_name category   test_typ